This is to find the nash equilibrium of pure strategies of n species and n resources. Each species has their metabolic pathways fixed and can switch between these pathways. Thus, the growth rate is first generated and the preference order can be changed. The gain of each strategy is determined by the net abundance of the species.

In [14]:
# initialization

import numpy as np
import math
import random
import itertools
from scipy.optimize import root
import copy
import time
# set resource number and amount
Nr = 2

# generate species pool: growth rate from gaussian; preference list; invasion order
# growth rates
Res = [1.0, 1.0]
Nb = Nr
Nl = math.factorial(Nr)
g_mean, g_var = 1, 0.2
growth_rate_list = np.exp(np.random.normal(g_mean, math.sqrt(g_var), (Nb, Nr)))
filt = np.heaviside(growth_rate_list, 0)
growth_rate_list = filt*growth_rate_list
#preference list
preference_list = list(itertools.permutations(range(Nr), Nr))
b0 = 1e-3 # density of bug when introduced/initial
b_threshold = 1e-6 # extinction density
c_threshold = 1e-9 # concentration threshold
# yields
yields_list = 0.5*np.ones([Nb, Nr]) # might need to be modified later?

# dilution parameters 
D = 1e-3
tau = 0 # lag
T_dilute = 10 # time interval between dilutions
dilute_to_steady = 10 # #(dilution) between invasions

In [176]:
def dilute(system, s1, s2): 
    pref_list = [preference_list[s1], preference_list[s2]]
    t_switch = 0
    while t_switch < T_dilute:
        #list of resource in use for each consumer
        use = [0 for i in range(Nb)]
        # list of consumers for each resource
        consumer = [[] for i in range(Nr)]
        for i in range(Nb):
            # if all are depleted, bug still uses its least preferred nutrient
            while system['res_available'][pref_list[i][use[i]]] < 1 and use[i] < Nr - 1:
                use[i] = use[i] + 1
            if system['bug_available'][i] > 0:
                consumer[pref_list[i][use[i]]].append(i)
        # find the earliest depleted resource
        t_dep = T_dilute - t_switch
        for i in range(Nr):
            if system['res_available'][i] > 0:
                def remain(t):  
                    # S = c - sum(B0(e^gt-1)/Y)                
                    return system['res_concentration'][i] - sum([system['bug_density'][j]*(math.exp(t*growth_rate_list[j][i]) - 1)/yields_list[j][i] for j in consumer[i]])
                    #print(consumer[0])
                # a little bit tricky here...
                t_i = root(remain, T_dilute + 1).x[0]
                if t_i < t_dep:
                    t_dep = t_i
        # update the system according to this t_dep
        t_switch = t_switch + t_dep
        temp_bug_density = [i for i in system['bug_density']]
        for i in range(Nb):
            if system['res_available'][pref_list[i][use[i]]] > 0:
                temp_bug_density[i] = system['bug_density'][i]*math.exp(growth_rate_list[i][pref_list[i][use[i]]]*t_dep)
        for i in range(Nr):
            if system['res_available'][i] > 0:
                system['res_concentration'][i] = system['res_concentration'][i] - sum([system['bug_density'][j]*(math.exp(t_dep*growth_rate_list[0][i]) - 1)/yields_list[j][i] for j in consumer[i]])
                if system['res_concentration'][i] < c_threshold: 
                    system['res_available'][i] = 0
        for i in range(Nb):
            system['bug_density'][i] = temp_bug_density[i]
    #print(system)
    #print("diluted once")
    return system
        
def move_to_new(system):
    system['bug_density'] = [(i*D > b_threshold)*i*D for i in system['bug_density']]
    system['bug_available'] = [1*(system['bug_density'][i] > b_threshold) for i in range(Nb)]
    system['res_concentration'] = [(system['res_concentration'][i]*D + Res[i]) / (D + 1) for i in range(Nr)]
    system['res_available'] = [1*(system['res_concentration'][i] > c_threshold) for i in range(Nr)]
    return system

def isNE(s1, s2, pay1, pay2):
    for i in range(Nl):
        if pay1[i, s2] > pay1[s1, s2]: return False
    for j in range(Nl):
        if pay2[s1, j] > pay2[s1, s2]: return False
    return True

In [177]:
# generate the matrices

pay1 = np.zeros([2, 2])
pay2 = np.zeros([2, 2])
for s1 in range(Nl):
    for s2 in range(Nl):
        system = {'res_available': [1, 1], 'res_concentration': [1.0, 1.0], 'bug_available': [1, 1], 'bug_density': [b0, b0]}
        for i in range(dilute_to_steady):
            system = dilute(system, s1, s2)
            system = move_to_new(system)
        system = dilute(system, s1, s2)
        pay1[s1, s2] = system['bug_density'][0]
        pay2[s1, s2] = system['bug_density'][1]

# find NE
NE = []
for s1 in range(Nl):
    for s2 in range(Nl):             
        if isNE(s1, s2, pay1, pay2) == True:
            NE.append([s1, s2])

IndexError: list index out of range

In [31]:
system = {'res_available': [1, 1], 'res_concentration': [1.0, 1.0], 'bug_available': [1, 1], 'bug_density': [b0, b0]}
system = dilute(system, 0, 0)
system

{'res_available': [0, 0],
 'res_concentration': [-0.884273725657992, 8.499901202807079e-10],
 'bug_available': [1, 1],
 'bug_density': [0.941950476227385, 0.07864458719606757]}

In [36]:
growth_rate_list

array([[4.60391516, 3.16380169],
       [2.54149674, 4.42412363]])

In [35]:
NE

[[0, 1]]

In [33]:
pay1

array([[1.000001  , 0.83382907],
       [1.000001  , 0.        ]])

In [34]:
pay2

array([[0.        , 0.39669544],
       [0.        , 1.6027205 ]])

The above is the case for 2 species. Now try 3 species and 3 resources. 

In [186]:
# initialization

import numpy as np
import math
import random
import itertools
from scipy.optimize import root, fsolve
import copy
import time

Res = [1.0, 1.0, 1.0]
Nr = 3
Nb = Nr
Nl = math.factorial(Nr)
g_mean, g_var = 1, 0.2
#growth_rate_list = np.exp(np.random.normal(g_mean, math.sqrt(g_var), (Nb, Nr)))
growth_rate_list = [np.array([9.20247937, 2.12018593, 1.08945719]),
  np.array([2.25233966, 8.45170172, 1.88723342]),
  np.array([4.78820702, 1.51485071, 9.48018625])]
filt = np.heaviside(growth_rate_list, 0)
growth_rate_list = filt*growth_rate_list
#preference list
preference_list = list(itertools.permutations(range(Nr), Nr))
b0 = 1e-3 # density of bug when introduced/initial
b_threshold = 1e-6 # extinction density
c_threshold = 1e-9 # concentration threshold
# yields
yields_list = 0.5*np.ones([Nb, Nr]) # might need to be modified later?

# dilution parameters 
D = 1e-3
tau = 0 # lag
T_dilute = 10 # time interval between dilutions
dilute_to_steady = 10 # #(dilution) between invasions

In [200]:
def dilute(system, s1, s2, s3): 
    pref_list = [preference_list[s1], preference_list[s2], preference_list[s3]]
    t_switch = 0
    while t_switch < T_dilute:
        #list of resource in use for each consumer
        use = [0 for i in range(Nb)]
        # list of consumers for each resource
        consumer = [[] for i in range(Nr)]
        for i in range(Nb):
            # if all are depleted, bug still uses its least preferred nutrient
            while system['res_available'][pref_list[i][use[i]]] < 1 and use[i] < Nr - 1:
                use[i] = use[i] + 1
            if system['bug_available'][i] > 0:
                consumer[pref_list[i][use[i]]].append(i)
        # find the earliest depleted resource
        t_dep = T_dilute - t_switch
        for i in range(Nr):
            if system['res_available'][i] > 0:
                def remain(t):  
                    # S = c - sum(B0(e^gt-1)/Y)   
                    return system['res_concentration'][i] - sum([system['bug_density'][j]*(math.exp(t*growth_rate_list[j][i]) - 1)/yields_list[j][i] for j in consumer[i]])
                # a little bit tricky here...
                t_i = root(remain, T_dilute + 1).x[0]
                if t_i < t_dep:
                    t_dep = t_i
        # update the system according to this t_dep
        t_switch = t_switch + t_dep
        temp_bug_density = [i for i in system['bug_density']]
        for i in range(Nb):
            if system['res_available'][pref_list[i][use[i]]] > 0:
                temp_bug_density[i] = system['bug_density'][i]*math.exp(growth_rate_list[i][pref_list[i][use[i]]]*t_dep)
        for i in range(Nr):
            if system['res_available'][i] > 0:
                system['res_concentration'][i] = system['res_concentration'][i] - sum([system['bug_density'][j]*(math.exp(t_dep*growth_rate_list[j][i]) - 1)/yields_list[j][i] for j in consumer[i]])
                #print(system['res_concentration'])
                if system['res_concentration'][i] < c_threshold: 
                    system['res_available'][i] = 0
        for i in range(Nb):
            system['bug_density'][i] = temp_bug_density[i]
    #print(system)
    #print("diluted once")
    return system
        
def move_to_new(system):
    system['bug_density'] = [(i*D > b_threshold)*i*D for i in system['bug_density']]
    system['bug_available'] = [1*(system['bug_density'][i] > b_threshold) for i in range(Nb)]
    system['res_concentration'] = [(system['res_concentration'][i]*D + Res[i]) / (D + 1) for i in range(Nr)]
    system['res_available'] = [1*(system['res_concentration'][i] > c_threshold) for i in range(Nr)]
    return system

def isNE(s1, s2, s3, pay1, pay2, pay3):
    for i in range(Nl):
        if pay1[i, s2, s3] > pay1[s1, s2, s3]: return False
    for i in range(Nl):
        if pay2[s1, i, s3] > pay2[s1, s2, s3]: return False
    for i in range(Nl):
        if pay3[s1, s2, i] > pay3[s1, s2, s3]: return False
    return True

def test(growth_rate_list, system, s1, s2, s3):
    for i in range(dilute_to_steady):
        system = dilute(system, s1, s2, s3)
        system = move_to_new(system)
    system = dilute(system, s1, s2, s3)
    return system

In [201]:
# generate the matrices
start = time.time()

pay1 = np.zeros((Nl, Nl, Nl))
pay2 = np.zeros((Nl, Nl, Nl))
pay3 = np.zeros((Nl, Nl, Nl))
for s1 in range(Nl):
    for s2 in range(Nl):
        print('check')
        for s3 in range(Nl):
            system = {'res_available': [1, 1, 1], 'res_concentration': [1.0, 1.0, 1.0], 'bug_available': [1, 1, 1], 'bug_density': [b0, b0, b0]}
            system = test(growth_rate_list, system, s1, s2, s3)
            pay1[s1, s2, s3] = system['bug_density'][0]
            pay2[s1, s2, s3] = system['bug_density'][1]
            pay3[s1, s2, s3] = system['bug_density'][2]

# find NE
NE = []
for s1 in range(Nl):
    for s2 in range(Nl):    
        for s3 in range(Nl):
            if isNE(s1, s2, s3, pay1, pay2, pay3) == True:
                NE.append([s1, s2, s3])
                
end = time.time()
print(end-start)

check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
7.748397588729858


In [202]:
growth_rate_list

array([[9.20247937, 2.12018593, 1.08945719],
       [2.25233966, 8.45170172, 1.88723342],
       [4.78820702, 1.51485071, 9.48018625]])

In [208]:
system = {'res_available': [1, 1, 1], 'res_concentration': [1.0, 1.0, 1.0], 'bug_available': [1, 1, 1], 'bug_density': [b0, b0, b0]}
system = test(growth_rate_list, system, 0, 3, 4)
system

{'res_available': [0, 0, 0],
 'res_concentration': [-7.216449660063518e-16, 0.0, -1.3100631690576847e-14],
 'bug_available': [1, 1, 1],
 'bug_density': [0.5028159191525747, 0.33340695305350226, 0.6637786277954301]}

In [204]:
NE

[[0, 2, 4], [0, 3, 4], [1, 2, 4], [1, 3, 4]]

In [149]:
system = move_to_new(system)
system = dilute(system, 0, 1, 3)
system

{'res_available': [0, 0, 0],
 'res_concentration': [2.220446049250313e-16,
  -1.1102230246251565e-16,
  -3.1308289294429414e-14],
 'bug_available': [1, 0, 0],
 'bug_density': [1.5000015000015159, 0.0, 0.0]}

In [122]:
system = move_to_new(system)
system

{'res_available': [1, 1, 1],
 'res_concentration': [0.9981952201378186,
  0.9990009990015898,
  0.9988575397651553],
 'bug_available': [0, 1, 0],
 'bug_density': [0.0, 0.002684614521340522, 0.0]}

In [163]:
preference_list

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]